In [2]:
import numpy as np
import os
import pywt
import mne
import xml.etree.ElementTree as ET
import math
import re
import glob

def wavelet_denoise(signal, wavelet='sym6', level=5, mode='soft', boundary_mode='symmetric'):
    if signal.ndim != 2:
        raise ValueError("输入必须是二维数组")
    denoised = np.zeros_like(signal)
    for i in range(signal.shape[0]):
        ch = signal[i]
        max_lvl = pywt.dwt_max_level(len(ch), pywt.Wavelet(wavelet))
        actual_lvl = min(level, max_lvl)
        coeffs = pywt.wavedec(ch, wavelet, level=actual_lvl, mode=boundary_mode)
        sigma = np.median(np.abs(coeffs[-1])) / 0.6745
        threshold = sigma * np.sqrt(2 * np.log(len(ch)))
        coeffs_thresh = [coeffs[0]] + [pywt.threshold(c, threshold, mode=mode) for c in coeffs[1:]]
        ch_denoised = pywt.waverec(coeffs_thresh, wavelet, mode=boundary_mode)
        denoised[i, :len(ch_denoised)] = ch_denoised[:len(ch)]
    return denoised

def standardize_signal(signal):
    mean = np.mean(signal)
    std = np.std(signal)
    return (signal - mean) / (std if std > 0 else 1e-6)

def preprocess_channel(signal, ch_name, fs):
    if ch_name in ['EEG1', 'EEG3']:
        return wavelet_denoise(signal[np.newaxis, :])[0]
    elif ch_name == 'EOG-L':
        return wavelet_denoise(signal[np.newaxis, :], level=3)[0]
    elif ch_name in ['EMG', 'Pres']:
        return signal
    else:
        raise ValueError(f"未知通道: {ch_name}")

def preprocess_all_channels(data, ch_names, fs, standardize=True):
    processed = np.zeros_like(data)
    for i, ch in enumerate(ch_names):
        sig = preprocess_channel(data[i], ch, fs)
        if standardize and ch in ['EEG1', 'EEG3', 'EOG-L']:
            sig = standardize_signal(sig)
        processed[i] = sig
    return processed

def generate_stage_annotations(xml_path, epoch_length=30, total_duration=None):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    events = []
    for ev in root.iter('ScoredEvent'):
        if ev.findtext('EventType') != 'Stages|Stages':
            continue
        concept = ev.findtext('EventConcept', '')
        parts = concept.split('|')
        if len(parts) < 2 or not parts[1].isdigit():
            continue
        label = int(parts[1])
        label = 4 if label == 5 else label
        try:
            start = float(ev.findtext('Start'))
            duration = float(ev.findtext('Duration'))
        except:
            continue
        if duration <= 0:
            continue
        events.append({'start': start, 'end': start + duration, 'label': label})
    max_time = total_duration or max([e['end'] for e in events] + [0])
    total_epochs = math.ceil(max_time / epoch_length)
    annotations = [{'epoch': i, 'label': 0} for i in range(total_epochs)]
    for e in events:
        s = int(e['start'] // epoch_length)
        e_ = int((e['end'] - 1e-9) // epoch_length)
        for i in range(s, min(e_ + 1, total_epochs)):
            annotations[i]['label'] = e['label']
    return annotations

def process_psg_with_segmentation(edf_path, xml_path, output_dir, channels=None, epoch_length=30, resample_rate=100):
    try:
        raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)
        if resample_rate:
            raw.resample(resample_rate)
        ch_names = raw.ch_names
        if channels:
            existing = [ch for ch in channels if ch in ch_names]
            missing = set(channels) - set(existing)
            if missing:
                raise ValueError(f"缺失通道: {missing}")
            raw.pick_channels(existing)
        fs = raw.info['sfreq']
        data = raw.get_data()
        data = preprocess_all_channels(data, raw.ch_names, fs, standardize=True)
        stage_anns = generate_stage_annotations(xml_path)
        samples_per_epoch = int(epoch_length * fs)
        for i, ann in enumerate(stage_anns):
            seg = data[:, i * samples_per_epoch:(i + 1) * samples_per_epoch]
            label = ann['label']
            fname = os.path.join(output_dir, f"ex{i:04d}_{label}.npy")
            np.save(fname, seg)
        print(f"[完成] {edf_path} 处理完毕，共保存 {len(stage_anns)} 个片段。")
    except Exception as e:
        print(f"[错误] 处理 {edf_path} 时失败：{e}")

# ==== 主处理流程 ====
edf_dir = "/data02/mesa/edfs"
xml_dir = "/data02/mesa/annotations-events-nsrr"
output_base_dir = "/data02/mesa/staging"

n_files = 3000
channels = ['EEG1', 'EEG3', 'EOG-L', 'EMG', 'Pres']
resample_rate = 100  # 降采样率

edf_files = glob.glob(os.path.join(edf_dir, "mesa-sleep-*.edf"))
edf_files.sort(key=lambda x: int(re.search(r'mesa-sleep-(\d+)\.edf', os.path.basename(x)).group(1)))

if isinstance(n_files, int):
    edf_files = edf_files[1128:n_files]
    print(f"准备处理前 {n_files} 个文件")

total_files = len(edf_files)
processed_count = 0
failed_count = 0

for edf_path in edf_files:
    try:
        base_name = os.path.basename(edf_path)
        file_id = re.search(r'mesa-sleep-(\d+)\.edf', base_name).group(1)
        xml_filename = f"mesa-sleep-{file_id}-nsrr.xml"
        xml_path = os.path.join(xml_dir, xml_filename)
        if not os.path.exists(xml_path):
            print(f"[SKIP] 缺失标注文件: {xml_filename}")
            continue
        output_dir = os.path.join(output_base_dir, file_id)
        os.makedirs(output_dir, exist_ok=True)
        processed_count += 1
        print(f"\n{'='*50}")
        print(f"正在处理文件 ({processed_count}/{total_files}): {base_name}")
        process_psg_with_segmentation(
            edf_path=edf_path,
            xml_path=xml_path,
            output_dir=output_dir,
            channels=channels,
            epoch_length=30,
            resample_rate=resample_rate
        )
    except Exception as e:
        print(f"[ERROR] 处理文件 {base_name} 失败: {str(e)}")
        failed_count += 1
        continue

print("\n批量处理完成！处理情况：")
print(f"总文件数: {total_files}")
print(f"成功处理: {processed_count}")
print(f"失败文件: {failed_count}")


准备处理前 3000 个文件

正在处理文件 (1/928): mesa-sleep-3822.edf
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


/tmp/ipykernel_1468737/3565405909.py:92: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw.pick_channels(existing)


[完成] /data02/mesa/edfs/mesa-sleep-3822.edf 处理完毕，共保存 1199 个片段。

正在处理文件 (2/928): mesa-sleep-3823.edf
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


/tmp/ipykernel_1468737/3565405909.py:92: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw.pick_channels(existing)


[完成] /data02/mesa/edfs/mesa-sleep-3823.edf 处理完毕，共保存 1379 个片段。

正在处理文件 (3/928): mesa-sleep-3833.edf
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


...

[完成] /data02/mesa/edfs/mesa-sleep-6810.edf 处理完毕，共保存 1319 个片段。

正在处理文件 (927/928): mesa-sleep-6811.edf
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


/tmp/ipykernel_1468737/3565405909.py:92: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw.pick_channels(existing)


[完成] /data02/mesa/edfs/mesa-sleep-6811.edf 处理完毕，共保存 1200 个片段。

正在处理文件 (928/928): mesa-sleep-6812.edf
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


/tmp/ipykernel_1468737/3565405909.py:92: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw.pick_channels(existing)


[完成] /data02/mesa/edfs/mesa-sleep-6812.edf 处理完毕，共保存 1079 个片段。

批量处理完成！处理情况：
总文件数: 928
成功处理: 928
失败文件: 0
